In [2]:
import pandas as pd

In [4]:
df=pd.read_csv("/Users/avantikagupta/spotify_recommender_system /spotify_millsongdata.csv")


In [5]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [6]:
df.head(5)

,artist,song,text
0,Def Leppard,Glad I'm Alive,Hello and welcome my friend \r\nIt's good to ...
1,Chicago,Long Lost Friend,The moment that we met \r\nThere was somethin...
2,Pat Benatar,Purgatory,We will be the forgotten ones \r\nThey will s...
3,Imperials,Praise The Lord,When you're up against a struggle that shatter...
4,Luther Vandross,Hustle,I love when the day is new \r\nWe get to star...


In [7]:
df.tail(5)

,artist,song,text
4995,Beautiful South,My Book,This is my life and this is how it reads \r\n...
4996,U2,If God Will Send His Angels,Nobody else here baby \r\nNo one here to blam...
4997,Modern Talking,Ready For The Victory,"Ready for the party, 10-9-8-7-6-5-4-3-2-1 go ..."
4998,Conway Twitty,Baby's Gone,Baby's Gone \r\nMy hand is trembling \r\nAs ...
4999,Linda Ronstadt,Heat Wave,Whenever I'm with him \r\nSomething inside st...


In [8]:
df.isnull().sum()

artist    0
song      0
text      0
dtype: int64

In [9]:
df['text'][0]

"Hello and welcome my friend  \r\nIt's good to see you again  \r\nDo you know anything new  \r\nTell me is this really you  \r\nHey, what's the story in LA  \r\nI haven't been there today  \r\nIt's time we went for a ride  \r\nCome on and step right inside  \r\n  \r\nToo many people have waited  \r\nToo many problems created  \r\nToo many jitterbugs not enough jive  \r\nI'm tired of sitting here waiting  \r\nWe should be out there creating  \r\nIt's times like this that I'm glad I'm alive  \r\n  \r\nDarkness our sole destination  \r\nDaylight our only intention  \r\nWhite lightning brightens the sky  \r\nCan't tell we're flying so high  \r\nRest now and don't be afraid  \r\nCan't you see we've got it made  \r\nJust cause I'm not going far  \r\nThat doesn't mean that we can't reach the stars  \r\n  \r\nToo many people have waited  \r\nToo many problems created  \r\nToo many jitterbugs not enough jive  \r\nI'm tired of sitting here waiting  \r\nWe should be out there creating  \r\nIt's t

In [10]:
df.shape


(5000, 3)

Text Cleaning/Text Processing

In [11]:
df['text'].str.lower().replace(r'\w\s','').replace(r'\n',' ',regex=True)

0       hello and welcome my friend  \r it's good to s...
1       the moment that we met  \r there was something...
2       we will be the forgotten ones  \r they will sh...
3       when you're up against a struggle that shatter...
4       i love when the day is new  \r we get to start...
                              ...                        
4995    this is my life and this is how it reads  \r f...
4996    nobody else here baby  \r no one here to blame...
4997    ready for the party, 10-9-8-7-6-5-4-3-2-1 go  ...
4998    baby's gone  \r my hand is trembling  \r as th...
4999    whenever i'm with him  \r something inside sta...
Name: text, Length: 5000, dtype: object

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

In [13]:
stemmer = PorterStemmer()

In [14]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/avantikagupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
tokenizer = TreebankWordTokenizer()

def token(txt):
    tokens = tokenizer.tokenize(txt)
    a = [stemmer.stem(w) for w in tokens]
    return " ".join(a)

print(token("you are beautiful, beauty"))


you are beauti , beauti


In [16]:
df['text'].apply(lambda x: token(x))

0       hello and welcom my friend it 's good to see y...
1       the moment that we met there wa someth so fami...
2       we will be the forgotten one they will shake t...
3       when you 're up against a struggl that shatter...
4       i love when the day is new we get to start all...
                              ...                        
4995    thi is my life and thi is how it read for ever...
4996    nobodi els here babi no one here to blame no o...
4997    readi for the parti , 10-9-8-7-6-5-4-3-2-1 go ...
4998    babi 's gone my hand is trembl as the key unlo...
4999    whenev i 'm with him someth insid start to bur...
Name: text, Length: 5000, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
tfid=TfidfVectorizer(analyzer='word', stop_words='english')

In [21]:
matrix=tfid.fit_transform(df['text'])

In [22]:
similar=cosine_similarity(matrix)

In [23]:
similar[0]

array([1.        , 0.05202528, 0.04704923, ..., 0.04615934, 0.01929627,
       0.01399306], shape=(5000,))

In [25]:
df[df['song']=='My Book'].index[0]

np.int64(4995)

Recommender Function

In [26]:
def recommender(song_name):
    idx= df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x:x[1])
    song=[]
    for s_id in distance[1:20]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [27]:
recommender("My Book")

['Cold Hearted',
 'Tonight',
 'Fine Fine Fine',
 'A Fine Romance',
 "You're Ever So Inviting",
 'So Fine',
 'Stay',
 'Me, Myself, And I',
 'Strange Design',
 "I'm Gonna Start Today",
 'Terry',
 "Death's Door",
 'Shamrock Diaries',
 'Bluebird Island',
 'My Arena',
 'Half As Much',
 'I Know Him So Well',
 "Six O'clock News",
 "It's Only Life"]

In [32]:
import pickle

# Save similarity matrix
pickle.dump(similar, open("similarity.pkl", "wb"))

# Save dataframe
pickle.dump(df, open("df.pkl", "wb"))


In [34]:
import os
os.listdir()


['algotrading_env',
 'Avantika Gupta.pdf',
 'Discord.dmg',
 '.zshrc.save',
 'c++ programs',
 '.config',
 'Music',
 'Miniconda3-latest-MacOSX-arm64.sh',
 '.hello.java.swp',
 'flask maybe',
 'ISL-Recognition-System',
 '.cursor',
 'jcef_35272.log',
 'PythonPiano',
 'run-through-cmd-line.py',
 '.zprofile.pysave',
 'pacman',
 'similarity.pkl',
 '.condarc',
 'medical 2024.pdf',
 'c++',
 'rohan',
 'Untitled1.ipynb',
 "cursor-complain'.mov",
 '.DS_Store',
 'helper_functions.py',
 'nltk_data',
 'jupyter.tar.gz',
 'Discord (2).dmg',
 'ecommerce',
 'get-pip.py',
 'mysql.dmg',
 'requirements.txt',
 '.CFUserTextEncoding',
 '.xonshrc',
 'python',
 'latest.code-workspace',
 'locofy',
 'movie recom',
 'Main Summary-20250311T105458Z-001.zip',
 'images',
 '.bashrc',
 'Untitled.ipynb',
 '.zshrc',
 'spotify_recommender_system ',
 '2024 code',
 '.streamlit',
 '.local',
 '.psql_history',
 '.mongodb',
 'Person.class',
 'Pictures',
 'LeaveManagementSystemAvantika',
 'request.csv',
 'googleresume.jpeg',
 'pacm